<a href="https://colab.research.google.com/github/ansu0122/stream-and-detect/blob/main/trainer/mlflow-yolo8-databricks.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install boto3 s3fs mlflow ultralytics databricks-sdk databricks-cli

In [2]:
import os
import logging
import yaml
import s3fs
import boto3
import numpy as np
import torch
import mlflow
from mlflow.exceptions import MlflowException
from mlflow.models import infer_signature
from ultralytics import settings
from ultralytics import YOLO

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!cp /content/drive/MyDrive/Colab/data/yolo-trafic.zip /content/
!mkdir -p /content/datasets
!unzip /content/yolo-trafic.zip -d /content/datasets/
!rm /content/yolo-trafic.zip

# move to datasets subfolder

In [5]:
os.environ['AWS_REGION'] = 'eu-central-1'
os.environ['AWS_ACCESS_KEY_ID'] = ''
os.environ['AWS_SECRET_ACCESS_KEY'] = ''

In [6]:
os.environ['DATABRICKS_HOST'] = 'https://community.cloud.databricks.com/'
os.environ['DATABRICKS_USERNAME'] = ''
os.environ['DATABRICKS_PASSWORD'] = ''

In [7]:
logging.getLogger("mlflow").setLevel(logging.DEBUG)

In [8]:
artifact_uri = f"s3://stream-n-detect/models"
DATA_YAML_PATH = 'datasets/data.yaml'
fs = s3fs.S3FileSystem()
settings.update({"dvc": False, "mlflow": False})
print(settings)

{'settings_version': '0.0.4', 'datasets_dir': '/content/datasets', 'weights_dir': 'weights', 'runs_dir': 'runs', 'uuid': '569f3ba64b326db489132663f79cd37279811de477381b83ac131e6cdd129cbb', 'sync': True, 'api_key': '', 'openai_api_key': '', 'clearml': True, 'comet': True, 'dvc': False, 'hub': True, 'mlflow': False, 'neptune': True, 'raytune': True, 'tensorboard': True, 'wandb': True}


In [9]:
mlflow.set_tracking_uri(uri="databricks")
experiment_name = "/Users/andriysukh0912@gmail.com/yolov8n-detect"
try:
    experiment_id = mlflow.create_experiment(experiment_name, artifact_location=artifact_uri)
except MlflowException:
    experiment = mlflow.get_experiment_by_name(experiment_name)
    experiment_id = experiment.experiment_id

mlflow.set_experiment(experiment_name)

<Experiment: artifact_location='s3://stream-n-detect/models', creation_time=1718367076967, experiment_id='541998404243590', last_update_time=1718385700381, lifecycle_stage='active', name='/Users/andriysukh0912@gmail.com/yolov8n-detect', tags={'mlflow.experiment.sourceName': '/Users/andriysukh0912@gmail.com/yolov8n-detect',
 'mlflow.experimentType': 'MLFLOW_EXPERIMENT',
 'mlflow.ownerEmail': 'andriysukh0912@gmail.com',
 'mlflow.ownerId': '3490293259147743'}>

In [10]:
# load params
with open(r"params.yaml") as f:
    params = yaml.safe_load(f)

# load a pre-trained model
model = YOLO(params['model_type'])

100%|██████████| 6.23M/6.23M [00:00<00:00, 124MB/s]


In [11]:
cat /root/.config/Ultralytics/settings.yaml

settings_version: 0.0.4
datasets_dir: /content/datasets
weights_dir: weights
runs_dir: runs
uuid: 569f3ba64b326db489132663f79cd37279811de477381b83ac131e6cdd129cbb
sync: true
api_key: ''
openai_api_key: ''
clearml: true
comet: true
dvc: false
hub: true
mlflow: false
neptune: true
raytune: true
tensorboard: true
wandb: true


In [12]:
# train
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
results = model.train(
    data=DATA_YAML_PATH,
    imgsz=params['imgsz'],
    batch=params['batch'],
    epochs=params['epochs'],
    optimizer=params['optimizer'],
    lr0=params['lr0'],
    seed=params['seed'],
    pretrained=params['pretrained'],
    # name=params['name'],
    device = device,
    project = params['project_dir']
)

engine/trainer: task=detect, mode=train, model=yolov8n.pt, data=datasets/data.yaml, epochs=20, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=cuda, workers=8, project=models, name=train, exist_ok=False, pretrained=True, optimizer=SGD, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_width=None, format=torchscript, keras=False, optimize=False, int8=False, dyna

100%|██████████| 755k/755k [00:00<00:00, 25.4MB/s]


Overriding model.yaml nc=80 with nc=5

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

/usr/local/lib/python3.10/dist-packages/torch/nn/modules/conv.py:456: UserWarning: Plan failed with a cudnnException: CUDNN_BACKEND_EXECUTION_PLAN_DESCRIPTOR: cudnnFinalize Descriptor Failed cudnn_status: CUDNN_STATUS_NOT_SUPPORTED (Triggered internally at ../aten/src/ATen/native/cudnn/Conv_v8.cpp:919.)
  return F.conv2d(input, weight, bias, self.stride,


AMP: checks passed ✅


...

      20/20      2.24G     0.9256     0.5383     0.9242        171        640: 100%|██████████| 363/363 [02:31<00:00,  2.40it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:06<00:00,  2.75it/s]

                   all        549       6270      0.869      0.867      0.908      0.669



20 epochs completed in 0.895 hours.
Optimizer stripped from models/train/weights/last.pt, 6.2MB
Optimizer stripped from models/train/weights/best.pt, 6.2MB

Validating models/train/weights/best.pt...
Ultralytics YOLOv8.2.32 🚀 Python-3.10.12 torch-2.3.0+cu121 CUDA:0 (Tesla T4, 15102MiB)
Model summary (fused): 168 layers, 3006623 parameters, 0 gradients, 8.1 GFLOPs


                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 18/18 [00:18<00:00,  1.03s/it]


                   all        549       6270      0.869      0.867      0.908      0.669
               bicycle        189        250      0.914      0.884      0.921      0.724
                   bus         81        108      0.877      0.923       0.95      0.831
                   car        520       3842      0.876      0.939      0.963      0.753
             motorbike        331       1238      0.874      0.813      0.869      0.545
                person        196        832      0.803      0.778      0.838      0.491
Speed: 0.4ms preprocess, 3.6ms inference, 0.0ms loss, 5.5ms postprocess per image
Results saved to models/train


In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
project_dir = params['project_dir']
results = model.val(data=DATA_YAML_PATH, device=device, project = f'{project_dir}/val/')

YOLOv8n summary (fused): 168 layers, 3151904 parameters, 0 gradients, 8.7 GFLOPs


val: Scanning /Users/asukh/Work/VSCode/Python/stream-and-detect/trainer/data/valid/labels.cache... 549 images, 8 backgrounds, 0 corrupt: 100%|██████████| 549/549 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 35/35 [02:44<00:00,  4.71s/it]


                   all        549       6270      0.191      0.125       0.11     0.0678
                person        189        250      0.014        0.1    0.00631    0.00212
               bicycle         81        108          0          0          0          0
                   car        520       3842      0.538      0.504      0.504      0.317
            motorcycle        331       1238      0.402     0.0186     0.0418       0.02
              airplane        196        832          0          0   0.000563   0.000169
Speed: 1.4ms preprocess, 293.2ms inference, 0.0ms loss, 2.3ms postprocess per image
Results saved to models/val/val4


In [14]:
requirements_path = "requirements.txt"
with open(requirements_path, "w") as f:
    f.write("torch==2.3.1\n")
    f.write("cloudpickle==3.0.0\n")
    # Add other dependencies if needed

with mlflow.start_run(experiment_id=experiment_id):
    print(mlflow.get_artifact_uri())
    mlflow.log_params(params)
    mlflow.log_metric("precision", results.results_dict['metrics/precision(B)'])
    mlflow.log_metric("recall", results.results_dict['metrics/recall(B)'])
    mlflow.log_metric("fitness", results.fitness)
    mlflow.log_metric("inference", results.speed['inference'])
    mlflow.log_metric("loss", results.speed['loss'])
    mlflow.set_tag("training info", "yolo8n traffic data")
    signature = infer_signature(params=params)

    model_info = mlflow.pytorch.log_model(pytorch_model = model.model, artifact_path="models", signature=signature, pip_requirements=requirements_path)
    mlflow.log_artifact('models/train')


s3://stream-n-detect/models/758fccaeef674621abf2a4740f75f814/artifacts


2024/06/14 20:28:30 WARNING mlflow.utils.requirements_utils: Detected one or more mismatches between the model's dependencies and the current Python environment:
 - torch (current: 2.3.0+cu121, required: torch==2.3.1)
 - cloudpickle (current: 2.2.1, required: cloudpickle==3.0.0)
To fix the mismatches, call `mlflow.pyfunc.get_model_dependencies(model_uri)` to fetch the model's environment and install dependencies using the resulting environment file.


In [ ]:
loaded_model = mlflow.pyfunc.load_model(model_uri=model_info.model_uri)